- 아파치 스파크는 2012년에 map과 reduce 같은 고수준의 함수 연산자를 이용해 스트림 처리를 할 수 있는 스파크 스트리밍과 Dstream API를 도입함.
- Dstream API는 자바나 파이썬 객체에 대한 저수준 연산만 지원하기 때문에 고수준의 최적화 기법을 활용하는 데 한계가 생김.
- 2016년에 풍부한 최적화 기법을 지원하고 DataFrame이나 DataSet코드와 쉽게 통합할 수 있는 신규 스트리밍 API를 DataFrame 기반으로 구현함. -> 2.2 버전부터 안정화됨

## 20.1 스트림 처리란
- 스트림 처리는 신규 데이터를 끊임없이 처리해 결과를 만들어내는 행위
- 입력 데이터(스트림 처리 시스템에 도착한 일련의 이벤트)는 무한하고, 시작과 끝을 사전에 정의하지 않음
- 스트리밍 애플리케이션은 이벤트 스티림이 도착한 후 다양한 쿼리 연산 수행

<br>
- 스트림 처리는 고정된 입력 데이터셋을 다루는 배치 처리와 비교 가능
- 고정된 데이터는 일반적으로 애플리케이션에서 발생한 이력성 이벤트를 모두 저장하는 데이터 웨어하우스의 대규모 데이터셋이 될 수 있음
- 배치 처리는 스트림 처리와 유사한 연산 쿼리를 쓰지만 한번만 만들어낸다. 
<br>
- 스트림 처리와 배치 처리는 실전에서 함께 사용되고 있음
- 연속형 애플리케이션은 스트리밍, 배치 그리고 대화형 작업으로 구성된 통합 애플리케이션

### 20.1.1 스트림 처리 사례

- 통보와 알림
- 실시간 리포트
- 증분형 ETL
- 실시긴 제공용 데이터 갱신
- 실시간 의사결정
- 온라인 머신러닝
  - 여러 사용자의 실시간 데이터와 이력 데이터를 조합해 모델을 학습

### 20.1.2 스트림 처리의 장점
- 배치 애플리케이션은 직관적이고 유지 보수와 개발이 비교적 단순함
  - 배치 형태로 데이터 처리하면 스트리밍 시스템보다 훨씬 높은 처리량을 얻을 수 있음
- 스트림 처리는 대기 시간이 짧음
- 자동으로 연산 결과의 증분을 생성하므로 반복적인 배치 작업보단 결과를 수정하는 데 더 효율적임
  - 배치 작업 사용 시 실행할 때마다 전체 데이터를 읽어 들이고 매번 24시간치 데이터를 계산해야함
  - 스트림 처리는 이전 연산의 상태를 기억하고 신규 데이터만 계산할 수 있음

### 20.1.3 스트림 처리의 과제
- 배치 처리를 사용하는 경우 단순하게 time 필드 순으로 모든 이벤트를 정렬하고, 2와 5 사이에 10이 있는지 확인하면 되지만, 스트리밍의 경우 각 이벤트를 개별적으로 수신하면서 2와 5에 대한 이벤트를 기억하고 그 사이에 10이 존재하는지 알아내기 위해 이벤트 간의 일부 상태를 유지해야함. 
- 스트림 자체를 기억하는게 매우 어려움

- 문제점 정리 
  - 애플리케이션 타임스탬프(이벤트 시간) 기준으로 순서가 뒤섞인 데이터 처리하기
  - 대규모의 상태 정보 유지하기
  - 높은 데이터 처리량 보장하기
  - 장애 상황에서도 정확히 한 번 처리하기
  - 부하 불균형과 뒤처진 서버 다루기
  - 이벤트에 빠르게 응답하기
  - 다른 저장소 시스템의 외부 데이터와 조인하기
  - 신규 이벤트 도착 시 출력 싱크의 갱신 방법 결정하기
  - 출력 시승템에 데이터 저장 시 트랜잭션 보장하기
  - 런타임에 비즈니스 로직 변경하기

## 20.2 스트림 처리의 핵심 설계 개념

### 20.2.1 레코드 단위 처리와 선언형 API
- 스트리밍 API를 설계하는 가장 쉬운 방법은 각 이벤트를 애플리케이션에 전달하고 사용자 코드에 반응하도록 만드는 것.
  - 아파치 스톰같은 여러 스트리밍 시스템에서 사용함
- 레코드 단위 처리 API를 사용하는 스트리밍 시스템은 애플리케이션 내부에서 여러 처리 파이프라인을 연결하는 기능만 제공함
  - 사용자가 직접 상태를 추적해야 함
  - 메모리 확보를 위한 상태 제거와 중복 처리 방지 등의 기능을 구현해야함
- 최신 스트리밍 시스템에서는 선언형 API를 제공함
  - 애플리케이션을 정의할 때 어떻게 신규 데이터를 처리하고 장애 상황에서 복구할지 지정하는 대신 무엇을 처리할지 지정함
  - DStream API에는 맵, 리듀스, 필터 같은 연산을 기반으로 하는 함수형 API를 가지고 있음
- 스파크의 구조적 스트리밍은 별도의 프로그래밍 없이 함수형 연산을 훨씬 효율적으로 처리할 수 있는 SQL 형태의 관계형 연산으로 변환시킴

### 20.2.2 이벤트 시간과 처리 시간
- 이벤트 시간 처리(원천 시스템에서 각 레코드에 기록한 타임스탬프를 기반으로 데이터를 처리하는 방식)를 지원할지 고려해야함.
- 처리 시간 기준 처리는 스트리밍 애플리케이션에 레코드가 도착한 시간을 기반으로 처리하는 방식
- 스트리밍 레코드는 순서가 뒤섞일 수 있다. 
- 이벤트 시간 처리 기반 애플리케이션 고려 요소
  - 시스템은 늦게 도착한 이벤트를 처리할 수 있도록 상태를 추적해야함 
  - 이벤트 시간이 속한 특정 시간 윈도우의 결과를 가장 적절하게 출력할 수 있는 시점을 결정해야함
  - 시스템이 해당 시점까지의 모든 입력 데이터를 수신했을 가능성이 높은 시기가 언제인지 결정해야함
  - 선언형 API에 이를 해결할 수 있는 것들이 있기 때문에 괜찮음

### 20.2.3 연속형 처리와 마이크로 배치 처리
- 연속형 처리 기반 시스템
  - 연속형 처리 기반의 시스템을 구성하는 각 노드는 다른 노드에서 전송하는 메시지를 끊임없이 수신하고 새로 갱신된 정보를 자신의 하위 노드로 전송함 (레코드별 데이터 처리)
  - 각 노드가 신규 메시지에 즉시 반응하기 때문에 전체 입력량이 비교적 적을 때 가장 빠르게 응답함
  - 레코드 단위 부하가 매우 크기 때문에 최대 처리량이 적다
  
- 마이크로 배치
  - 마이크로 배치 시스템은 입력 데이터를 작은 배치로 모의기 위해 대기하고, 배치 잡 실행 방식과 유사하게 다수의 분산 태스크를 이용해 각 배치를 병렬로 처리함
  - 배치 시스템의 최적화 기법을 사용할 수 있음
  - 더 적은 노드로 같은 양의 데이터를 처리할 수 있음
  - 부하 분산(워크로드 변화에 대응할 수 있도록 태스크 수를 늘리거나 줄이는 방식)을 동적으로 사용 가능
  - 배치를 모으기 위한 시간이 필요하기 때문에 약간의 지연 존재함



## 20.3 스파크 스트리밍 API

### 20.3.1 DStream API
- 많은 장점을 제공하는 구조적 테이블 개념의 DataFrame이나 Dataset과 달리 자바나 파이썬의 객체와 함수에 매우 의존적임. 따라서 스트림 처리 엔진이 제공하는 최적화 기법 적용 불가
- DStream API는 기본적으로 처리 시간을 기준으로 동작함. 이벤트 시간은 직접 구현해야함
- 마이크로 배치 형태로만 동작함

### 20.3.2 구조적 스트리밍 
- 구조적 API를 기반으로 한 고수준 스트리밍 API
- 스칼라, 자바, 파이썬, SQL을 사용해 구좆적 처리를 할 수 있는 모든 환경에서 사용가능
- 고수준 연산 기반의 선언형 API를 제공하고 구조적 데이터 모델을 기반으로 만들어짐
- 이벤트 시간 데이터 처리를 지원함 -> 모든 윈도우 연산에서 자동으로 이벤트 시간 처리를 지원함
- 단순한 스트림 처리 뿐만 아니라 스트리밍, 배치, 그리고 대화형 쿼리를 하나로 통합한 연속형 애플리케이션 쉽게 만들 수 있도록 설계됨 -> DataFrame과 같은 API 사용

